In [1]:
import re
import pandas as pd

In [2]:
train = pd.read_csv('data/email_best_send_time_train.csv')
test = pd.read_csv('data/email_best_send_time_test.csv')

In [4]:
import re

def rep_hi_person(text):
    t = re.split(r'Hi\,? ', text)
    if len(t) <= 1:
        return text
    
    person = re.split(r'[\!\,]', t[1])[0]
    if len(person.split(' ')) <= 2:
        text = text.replace(person, 'PERSON')
    return text


def rep_welcom_person(text):
    if text[:len('Welcome ')] == 'Welcome ':
        person = text[len('Welcome '):]
        person = re.split(r'[\!\,]', person)[0]
        if len(person.split(' ')) <= 2:
            text = text.replace(person, 'PERSON')
        return text
    
    t = text.split()
    if len(t)<2:
        return text
    if t[1].lower() == 'welcome':
        person = t[0]
        person = re.sub(r'\,', '', person)
        text = text.replace(person, 'PERSON')
    return text


def rep_first_person(text):
    t = text.split()
    if len(t)>0:
        if t[0][-1] == ',':
            person = t[0][:-1]
            if (person != 'Hi') and (person != 'PERSON'):
                text = text.replace(person, 'PERSON')
    return text


def rep_increase_productivity_person(text):
    if '--Increase your productivity by ' in text:
        person = text.split('--Increase your productivity by ')[0]
        text = text.replace(person, 'PERSON')
    return text


def rep_quick_question_person(text):
    if 'Quick question, ' in text:
        person = text.split('Quick question, ')[1]
        person = re.split(r'[\!\,\?]', person)[0]
        text = text.replace(person, 'PERSON')
    return text

def rep_1_person(text):
    for phr in ['Intelligent Salesforce integration with Outlook for ', 'Smart Salesforce integration with Inbox for ', 'Hey ', 
                'Long time no see, ', 'Greetings ', 'Thank you ', "Let's talk about partnership, ", 'Guided Selling demo for ', 
                'How are you, ']:
        #if phr in text:
        if phr  == text[:len(phr)]:
            person = text.split(phr)[1]
            person = re.split(r'[\!\,\?]', person)[0]
            if len(person.split(' ')) <= 2:
                text = text.replace(person, 'PERSON')
    return text

def rep_2_person(text):
    for phr in ['Noticing this ', 'Noticed this ']:
        if phr in text:
            person = text.split(phr)[1]
            person = person[:-3]
            if len(person.split(' ')) <= 2:
                text = text.replace(person, 'PERSON')
    return text


def rep_1_comp(text):
    for phr in ['Salesforce integration with Email & Calendar for ', 'Revenue Grid <> ', 'Integrating Salesforce and Email/Calendar for ' , 
                'Saubere Salesforce Daten mit Revenue Grid für ', 'Salesforce-Integration mit Email und Kalender für ']:
        if phr in text:
            person = text.split(phr)[1]
            text = text.replace(person, '_COMPANY_')
    return text


def rep_2_comp(text):
    for phr in ['PERSON, should I be Interpreting your Silence in respect to Integrating this to ']:
        if phr in text:
            person = text.split(phr)[1]
            person = person.split(' Salesforce Family?')[0]
            text = text.replace(person, '_COMPANY_')
    return text

def rep_3_comp(text):
    for phr in [' < > Revenue Grid']:
        if phr in text:
            person = text.split(phr)[0]
            text = text.replace(person, '_COMPANY_')
    return text


def rep_1_comp_pers(text):
    for phr in ['Revenue Inbox Trial Setup for ',]:
        if phr in text:
            person_comp = text.split(phr)[1]
            person_comp = person_comp.split(' - ')
            person = person_comp[0]
            comp = ' - '.join(person_comp[1:])
            text = text.replace(comp, '_COMPANY_')
            text = text.replace(person, 'PERSON')
            #text = text.replace(comp, '_COMPANY_')
    return text

def rep_2_comp_pers(text):
    for phr in ['Revenue Inbox - Smart Salesforce Integration for ',]:
        if phr in text:
            person_comp = text.split(phr)[1]
            person_comp = person_comp.split(' from ')
            person = person_comp[0]
            comp = ' from '.join(person_comp[1:])
            text = text.replace(comp, '_COMPANY_')
            text = text.replace(person, 'PERSON')
            #text = text.replace(comp, '_COMPANY_')
    return text




def rep_1_technology(text):
    for phr in ['Salesforce data engine that actually works for ']:
        #if phr in text:
        if phr  == text[:len(phr)]:
            person = text.split(phr)[1]
            person = re.split(r'[\!\,\?]', person)[0]
            text = text.replace(person, '_TECHNOLOGY_')
    return text

In [5]:
def clean_subject(train):
    train['Subject']  = train['Subject'].apply(lambda x: x[1:-1])
    train['Subject'] = train['Subject'].str.replace('\s+', ' ', regex=True).str.strip()

    train['subject_re'] = train['Subject'].str.lower().apply(lambda x: 're:' in x)
    train['Subject'] = train['Subject'].str.replace('RE:', ' ', regex=True).str.strip()
    train['Subject'] = train['Subject'].str.replace('Re:', ' ', regex=True).str.strip()
    train['Subject'] = train['Subject'].str.replace(' re:', ' ', regex=True).str.strip()
    train['Subject'] = train['Subject'].str.replace('^re:', ' ', regex=True).str.strip()
    train['Subject'] = train['Subject'].str.replace('\s+', ' ', regex=True).str.strip()
    
    train['Subject'] = train['Subject'].apply(rep_hi_person)
    train['Subject'] = train['Subject'].apply(rep_welcom_person)
    train['Subject'] = train['Subject'].apply(rep_first_person)
    train['Subject'] = train['Subject'].apply(rep_increase_productivity_person)
    train['Subject'] = train['Subject'].apply(rep_quick_question_person)


    train['Subject'] = train['Subject'].apply(rep_1_person)
    train['Subject'] = train['Subject'].apply(rep_2_person)
    train['Subject'] = train['Subject'].apply(rep_1_comp)
    train['Subject'] = train['Subject'].apply(rep_2_comp)
    train['Subject'] = train['Subject'].apply(rep_3_comp)
    train['Subject'] = train['Subject'].apply(rep_1_comp_pers)
    train['Subject'] = train['Subject'].apply(rep_2_comp_pers)
    train['Subject'] = train['Subject'].apply(rep_1_technology)
    
    train['Subject'] = train['Subject'].str.replace('PERSONtmas', 'Christmas')
    train['Subject'] = train['Subject'].str.replace('PERSONmas', 'Christmas')
    train['Subject'] = train['Subject'].str.strip()
    
    train['clean_subject'] = train['Subject']
    train['super_clean_subject'] = train['Subject'].apply(lambda x: re.sub(r'\W', '', x)).str.lower().apply(lambda x: x[:30])
    return train

In [6]:
train = clean_subject(train)
test = clean_subject(test)
train[['MailID', 'subject_re', 'clean_subject', 'super_clean_subject']].to_csv('train_clean_subject.csv', index=False)
test[['MailID', 'subject_re', 'clean_subject', 'super_clean_subject']].to_csv('test_clean_subject.csv', index=False)

In [7]:
train

,MailID,Subject,MailBoxID,ContactID,TimeZone,SentOn,Opened,subject_re,clean_subject,super_clean_subject
0,C278C72C-63D6-4D42-B5A6-455FA2C80D7C,Cold emails not converting?,9F1BAD1D-0370-4200-0045-08D70B884EBA,8977EFF2-DCAD-4E29-B29C-BE55449D24A2,"""(UTC+00:00) Dublin, Edinburgh, Lisbon, London""",9/9/20 11:34,0,False,Cold emails not converting?,coldemailsnotconverting
1,F5CA7090-9A73-4197-9923-C094B140C4D1,Webnar: Cold emails not converting?,7FCE4E75-9A26-4EE6-7806-08D6CEFA3BFA,22A565BC-DC52-4CF5-8852-5C343F477F54,"""(UTC-06:00) Central Time (US & Canada)""",9/9/20 11:35,0,False,Webnar: Cold emails not converting?,webnarcoldemailsnotconverting
2,B4837C83-9425-4C77-907E-E3B318A5F0F0,Cold emails not converting?,9F1BAD1D-0370-4200-0045-08D70B884EBA,00E3A137-053F-4838-AB6F-4A8F6F52969A,"""(UTC+00:00) Dublin, Edinburgh, Lisbon, London""",9/9/20 11:38,0,False,Cold emails not converting?,coldemailsnotconverting
3,204546C3-073D-418E-98BD-E5553B339686,Webnar: Cold emails not converting?,7FCE4E75-9A26-4EE6-7806-08D6CEFA3BFA,9A13D123-10AA-46BF-B5C9-7199074DDD71,"""(UTC-05:00) Eastern Time (US & Canada)""",9/9/20 11:40,0,False,Webnar: Cold emails not converting?,webnarcoldemailsnotconverting
4,F28E3C17-0000-4549-9153-5896688E7581,Webnar: Cold emails not converting?,7FCE4E75-9A26-4EE6-7806-08D6CEFA3BFA,FA770B61-2BB3-44EB-A064-0F80CAD8E753,"""(UTC-06:00) Central Time (US & Canada)""",9/9/20 11:43,0,False,Webnar: Cold emails not converting?,webnarcoldemailsnotconverting
...,...,...,...,...,...,...,...,...,...,...
100139,2EA3A30E-D8DB-44E3-A8B3-CA7D9BA2633C,Revenue Inbox | Sync integration for Salesforce,FBCA1245-7297-41A8-1AB0-08D7856F3B8E,FEB418B4-D3FF-4B84-8703-E8A71478C11C,"""(UTC+00:00) Dublin, Edinburgh, Lisbon, London""",7/1/21 11:48,1,True,Revenue Inbox | Sync integration for Salesforce,revenueinboxsyncintegrationfor
100140,114E23DD-5386-40FA-B7C8-AFD80628FE4D,Hi PERSON!,2B68B3E2-ADF7-4D0F-8F59-645C8CDF6EA6,748DFA05-47E9-4FB8-9EA3-0B5619CD6CB6,"""(UTC-05:00) Eastern Time (US & Canada)""",7/1/21 11:48,0,True,Hi PERSON!,hiperson
100141,6EE9027F-E9E5-476C-B1F4-F9A6433B5DDF,testy test,66C24EAB-51C7-4B5F-EFCA-08D6D3CCF2FF,E315C9A4-E5C4-484F-9278-6347BE3C2CD1,NaN,7/1/21 11:55,0,False,testy test,testytest
100142,DE170985-95AB-47B6-A077-B7B4BD6C6FF3,"PERSON, thanks for signing up to Revenue Inbox",FBCA1245-7297-41A8-1AB0-08D7856F3B8E,793680C5-DDFD-41A8-9194-61B1AFC9D9F2,"""(UTC+00:00) Dublin, Edinburgh, Lisbon, London""",7/1/21 11:55,0,True,"PERSON, thanks for signing up to Revenue Inbox",personthanksforsigninguptoreve


In [8]:
test['clean_subject'].value_counts()

Hi PERSON!                                               62165
761 rep-days returned                                     4619
Ramp time headache                                        3345
Hi, PERSON                                                3041
Revenue Grid                                              2166
                                                         ...  
Newsletter 2                                                 1
Noticed this PERSONie!                                       1
Ensure Data Integrity with Smart Auto-Capture Sudhir,        1
Ensure Data Integrity with Smart Auto-Capture Rutuja,        1
Noticed this PERSONrd!                                       1
Name: clean_subject, Length: 300, dtype: int64

In [9]:
from itertools import chain
all_words = pd.Series(list(chain(*train['clean_subject'].str.lower().str.split())))
all_words_vc = all_words.str.lower().value_counts()
print(all_words_vc.shape[0])
print(all_words_vc[all_words_vc > 1].shape[0])
print(all_words_vc[all_words_vc > 2].shape[0])
print(all_words_vc[all_words_vc > 5].shape[0])

all_words_vc[:30]

545
465
424
358


person!        63066
hi             62584
revenue         9619
for             8006
salesforce      6571
grid            6243
person,         6052
person          5022
rep-days        4679
returned        4679
761             4679
-               4406
inbox           4219
time            3947
to              3744
ramp            3448
headache        3448
you             3087
hi,             3037
_company_       2650
ai              2524
assist          2524
your            2175
integration     2136
sales           2071
platform        2017
engagement      1857
thank           1799
revenuegrid     1635
with            1572
dtype: int64